In [ ]:
import json

# Access parameters directly
bronze_output = ""

In [ ]:
# Parse the JSON string
output_data = json.loads(bronze_output)

# Access individual variables
start_date = output_data.get("start_date")
silver_adls = output_data.get("silver_adls")
bronze_adls = output_data.get("bronze_adls")

print(f"Start Date: {start_date}")
print(f"Silver ADLS: {silver_adls}")
print(f"Bronze ADLS: {bronze_adls}")

In [46]:
''' from datetime import date, timedelta

# Remove this before running Data Factory Pipeline
start_date = bronze_output[]

bronze_adls = "abfss://bronze@synonly.dfs.core.windows.net/"
silver_adls = "abfss://silver@synonly.dfs.core.windows.net/"
'''

StatementMeta(synonly, 15, 2, Finished, Available, Finished)

In [47]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

StatementMeta(synonly, 15, 3, Finished, Available, Finished)

In [48]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

StatementMeta(synonly, 15, 4, Finished, Available, Finished)

In [49]:
df

StatementMeta(synonly, 15, 5, Finished, Available, Finished)

DataFrame[geometry: struct<coordinates:array<double>,type:string>, id: string, properties: struct<code:string,detail:string,dmin:double,gap:double,ids:string,mag:double,magType:string,net:string,nst:bigint,place:string,rms:double,sig:bigint,sources:string,status:string,time:bigint,title:string,tsunami:bigint,type:string,types:string,updated:bigint,url:string>, type: string]

In [50]:
df.head()

StatementMeta(synonly, 15, 6, Finished, Available, Finished)

Row(geometry=Row(coordinates=[-116.1651667, 33.2081667, 7.72], type='Point'), id='ci40838207', properties=Row(code='40838207', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci40838207&format=geojson', dmin=0.2636, gap=186.0, ids=',ci40838207,', mag=1.13, magType='ml', net='ci', nst=11, place='8 km NNW of Ocotillo Wells, CA', rms=0.2, sig=20, sources=',ci,', status='automatic', time=1736639492580, title='M 1.1 - 8 km NNW of Ocotillo Wells, CA', tsunami=0, type='earthquake', types=',nearby-cities,origin,phase-data,scitech-link,', updated=1736639790427, url='https://earthquake.usgs.gov/earthquakes/eventpage/ci40838207'), type='Feature')

In [51]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

StatementMeta(synonly, 15, 7, Finished, Available, Finished)

In [52]:
df

StatementMeta(synonly, 15, 8, Finished, Available, Finished)

DataFrame[id: string, longitude: double, latitude: double, elevation: double, title: string, place_description: string, sig: bigint, mag: double, magType: string, time: bigint, updated: bigint]

In [53]:
df.head()

StatementMeta(synonly, 15, 9, Finished, Available, Finished)

Row(id='ci40838207', longitude=-116.1651667, latitude=33.2081667, elevation=7.72, title='M 1.1 - 8 km NNW of Ocotillo Wells, CA', place_description='8 km NNW of Ocotillo Wells, CA', sig=20, mag=1.13, magType='ml', time=1736639492580, updated=1736639790427)

In [54]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

StatementMeta(synonly, 15, 10, Finished, Available, Finished)

In [55]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

StatementMeta(synonly, 15, 11, Finished, Available, Finished)

In [56]:
df.head()

StatementMeta(synonly, 15, 12, Finished, Available, Finished)

Row(id='ci40838207', longitude=-116.1651667, latitude=33.2081667, elevation=7.72, title='M 1.1 - 8 km NNW of Ocotillo Wells, CA', place_description='8 km NNW of Ocotillo Wells, CA', sig=20, mag=1.13, magType='ml', time=datetime.datetime(2025, 1, 11, 23, 51, 32, 580000), updated=datetime.datetime(2025, 1, 11, 23, 56, 30, 427000))

In [57]:
# Save the transformed DataFrame to the Silver container
silver_data = f"{silver_adls}earthquake_events_silver/"

StatementMeta(synonly, 15, 13, Finished, Available, Finished)

In [58]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_data)

StatementMeta(synonly, 15, 14, Finished, Available, Finished)

In [59]:
mssparkutils.notebook.exit(silver_data)

StatementMeta(synonly, 15, 15, Finished, Available, Finished)

abfss://silver@synonly.dfs.core.windows.net/earthquake_events_silver/
